In [48]:
import requests

In [49]:
import geopandas as gpd
import pandas as pd

## Get State Data

In [5]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [6]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get COVID Data

In [45]:
def get_deaths(row: pd.Series):
    state_code = row["STUSPS"]
    url = f"https://jhucoronavirus.azureedge.net/api/v2/regions/us/{state_code.lower()}.json"
    r = requests.get(url)
    if r.status_code == 200:
        return r.json()["deaths"]["all"]
    else:
        print(row["NAME"])

In [46]:
states_df["deaths"] = states_df.apply(lambda row: get_deaths(row), axis=1)

## Get Population Data

In [50]:
state_populations = pd.read_excel(
    "data/NST-EST2023-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [51]:
state_populations_df = state_populations["NST-EST2023-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

## Merge Data

In [57]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "POPULATION", "geometry", "NAME", "deaths"]
]

In [58]:
states_with_population_df = states_with_population_df.dropna()

In [60]:
states_with_population_df["per_1000"] = states_with_population_df["deaths"] / (
    states_with_population_df["POPULATION"] / 1000
)
states_with_population_df["per_10k"] = states_with_population_df["deaths"] / (
    states_with_population_df["POPULATION"] / 10_000
)
states_with_population_df["per_100k"] = states_with_population_df["deaths"] / (
    states_with_population_df["POPULATION"] / 100000
)
states_with_population_df["per_500k"] = states_with_population_df["deaths"] / (
    states_with_population_df["POPULATION"] / 500_000
)
states_with_population_df["per_1m"] = states_with_population_df["deaths"] / (
    states_with_population_df["POPULATION"] / 1_000_000
)
states_with_population_df["per_capita"] = (
    states_with_population_df["POPULATION"] / states_with_population_df["deaths"]
)

In [61]:
states_with_population_df = states_with_population_df.to_crs(9311)

In [62]:
states_with_population_df.to_file("data/covid_deaths.gpkg")

In [63]:
states_with_population_df

,STUSPS,POPULATION,geometry,NAME,deaths,per_1000,per_10k,per_100k,per_500k,per_1m,per_capita
0,MS,2939690.0,"MULTIPOLYGON (((1109997.773 -1565578.875, 1111...",Mississippi,13370,4.548099,45.480986,454.809861,2274.049305,4548.098609,219.872102
1,NC,10835491.0,"MULTIPOLYGON (((2154808.027 -692253.992, 21554...",North Carolina,28432,2.623970,26.239697,262.396969,1311.984847,2623.969694,381.101963
2,OK,4053824.0,"POLYGON ((-268912.756 -936547.904, -268600.891...",Oklahoma,17972,4.433345,44.333449,443.334491,2216.672455,4433.344911,225.563321
3,VA,8715698.0,"MULTIPOLYGON (((2098153.247 -490400.387, 20981...",Virginia,23666,2.715330,27.153304,271.533043,1357.665215,2715.330430,368.279304
4,WV,1770071.0,"POLYGON ((1506482.53 -601393.056, 1506489.516 ...",West Virginia,7960,4.496995,44.969948,449.699475,2248.497377,4496.994753,222.370729
5,LA,4573749.0,"MULTIPOLYGON (((1079322.442 -1609024.878, 1079...",Louisiana,18766,4.102980,41.029799,410.297985,2051.489927,4102.979853,243.725301
6,MI,10037261.0,"MULTIPOLYGON (((1376542.504 -187676.574, 13766...",Michigan,42205,4.204832,42.048324,420.483237,2102.416187,4204.832374,237.821609
7,MA,7001399.0,"MULTIPOLYGON (((2420778.194 34524.899, 2421697...",Massachusetts,24333,3.475448,34.754483,347.544826,1737.724132,3475.448264,287.732668
8,ID,1964726.0,"POLYGON ((-1357016.205 78008.02, -1356337.908 ...",Idaho,5416,2.756618,27.566185,275.661848,1378.309240,2756.618480,362.763294
9,FL,22610726.0,"MULTIPOLYGON (((1997726.172 -1929592.915, 1997...",Florida,86850,3.841097,38.410974,384.109736,1920.548681,3841.097362,260.342268
